# Tre Torri BPR Calibration Notebook


This notebook helps estimate per-segment Bureau of Public Roads (BPR) model parameters (\u03b1, \u03b2, and capacity) by combining historical Google Routes travel-time samples with peak-hour vehicle counts collected in the field.

Steps covered here:
1. Load travel-time samples from `data/traffic_samples.jsonl`.
2. Load field counts from a separate CSV file.
3. Align counts with the corresponding travel-time windows.
4. Fit segment-level BPR parameters using non-linear least squares.
5. Export the calibrated coefficients so they can be copied into `src/segments.js` (`metadata.flowModel`).

> Tip: run the first two code cells (`pip install ...` and the imports) once per notebook kernel start. Subsequent execution should skip repeated installations.


In [ ]:
# Optional: install dependencies into the current environment
# Uncomment the next line if pandas/numpy/scipy/matplotlib/tqdm are missing.
# !pip install -q pandas numpy scipy matplotlib tqdm


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
plt.style.use('seaborn-v0_8')


## Configuration
Fill in the paths to the samples JSONL file (already part of this repository) and to the field counts CSV you collected. The counts file should include at least the following columns:

| column | description |
| --- | --- |
| `segmentId` | ID used in `src/segments.js` |
| `observationStart` | ISO datetime (UTC recommended) for the beginning of the counted interval |
| `observationEnd` | ISO datetime for the end of the counted interval |
| `observedVehicles` | Number of vehicles counted during the interval |
| `capacityGuessVph` *(optional)* | Prior capacity estimate in veh/h (e.g., lanes × laneCapacityVph) |

The notebook converts `observedVehicles` into veh/h using the interval duration. If you already have veh/h counts, keep `observedVehicles` equal to that value and set `observationEnd = observationStart + 1 hour`.


In [ ]:
PROJECT_ROOT = Path.cwd().resolve().parent
SAMPLES_PATH = PROJECT_ROOT / 'data' / 'traffic_samples.jsonl'
COUNTS_PATH = PROJECT_ROOT / 'data' / 'field_counts.csv'  # update with your counts file
OUTPUT_PATH = PROJECT_ROOT / 'data' / 'calibrated_flow_models.csv'

print(f'Project root: {PROJECT_ROOT}')
print(f'Samples path: {SAMPLES_PATH}')
print(f'Counts path:  {COUNTS_PATH}')
print(f'Output path:  {OUTPUT_PATH}')


## Load travel-time samples
The poller writes JSON Lines where every row contains `durationSeconds`, `staticDurationSeconds`, and optional enrichment fields. We focus on the ratio between live and free-flow travel times.


In [ ]:
samples = pd.read_json(SAMPLES_PATH, lines=True)
samples['requestedAt'] = pd.to_datetime(samples['requestedAt'], utc=True)
samples = samples.assign(
    durationSeconds=samples['durationSeconds'].astype(float),
    staticDurationSeconds=samples['staticDurationSeconds'].astype(float),
)
samples['durationRatio'] = np.where(
    (samples['staticDurationSeconds'] > 0) & (samples['durationSeconds'] > 0),
    samples['durationSeconds'] / samples['staticDurationSeconds'],
    np.nan,
)

print(f'Total samples loaded: {len(samples):,}')
samples.head()


## Load field counts
The counts CSV can contain multiple observations per segment. Ensure datetimes are parseable; they will be converted to timezone-aware timestamps (UTC).


In [ ]:
counts = pd.read_csv(COUNTS_PATH)
datetime_columns = [col for col in ['observationStart', 'observationEnd'] if col in counts.columns]
for col in datetime_columns:
    counts[col] = pd.to_datetime(counts[col], utc=True)

if 'observedVehicles' not in counts.columns:
    raise ValueError('Counts CSV must include an `observedVehicles` column (vehicle totals or veh/h).')

if 'capacityGuessVph' not in counts.columns:
    counts['capacityGuessVph'] = np.nan

if {'observationStart', 'observationEnd'}.issubset(counts.columns):
    duration_seconds = (counts['observationEnd'] - counts['observationStart']).dt.total_seconds()
    counts['observedFlowVph'] = np.where(
        duration_seconds > 0,
        counts['observedVehicles'] * 3600.0 / duration_seconds,
        counts['observedVehicles'],
    )
else:
    print('No observationStart/End columns found; treating `observedVehicles` as veh/h input.')
    counts['observedFlowVph'] = counts['observedVehicles']

print(f'Field counts loaded: {len(counts):,}')
counts.head()


## Align counts with travel-time samples
For each field observation, gather the travel-time samples collected within the same interval. You can adjust the `window_pad_minutes` parameter if you need to expand the matching window slightly.


In [ ]:
window_pad_minutes = 0  # increase (e.g. 5) if you want to include samples just outside the interval
pad = pd.Timedelta(minutes=window_pad_minutes)

def summarise_window(segment_id: str, start: pd.Timestamp, end: pd.Timestamp):
    mask = (samples['segmentId'] == segment_id)
    if pd.notna(start):
        mask &= samples['requestedAt'] >= (start - pad)
    if pd.notna(end):
        mask &= samples['requestedAt'] <= (end + pad)
    window = samples[mask]
    if window.empty:
        return None
    ratios = window['durationRatio'].dropna()
    if ratios.empty:
        return None
    return {
        'sampleCount': len(window),
        'meanRatio': ratios.mean(),
        'medianRatio': ratios.median(),
        'p90Ratio': ratios.quantile(0.90),
        'meanStaticSeconds': window['staticDurationSeconds'].mean(),
        'meanDurationSeconds': window['durationSeconds'].mean(),
    }

aggregated_rows = []
for idx, row in tqdm(counts.iterrows(), total=len(counts)):
    start = row.get('observationStart', pd.NaT)
    end = row.get('observationEnd', pd.NaT)
    summary = summarise_window(row['segmentId'], start, end)
    if summary is None:
        continue
    aggregated_rows.append({
        'segmentId': row['segmentId'],
        'observationStart': start,
        'observationEnd': end,
        'observedFlowVph': row['observedFlowVph'],
        'capacityGuessVph': row.get('capacityGuessVph'),
        **summary,
    })

aligned = pd.DataFrame(aggregated_rows)
print(f'Aligned observations: {len(aligned):,} (out of {len(counts):,} field rows)')
aligned.head()


## BPR calibration helpers
The calibration routine solves for \u03b1, \u03b2, and capacity by minimising the difference between observed travel-time ratios and the BPR model prediction:
\[ r = 1 + \alpha \left(\frac{v}{c}\right)^\beta \]
where `r` is the mean ratio, `v` the observed flow (veh/h), and `c` the capacity in veh/h.


In [ ]:
def calibrate_bpr(observations: pd.DataFrame, initial_alpha: float = 0.15, initial_beta: float = 4.0,
                  initial_capacity: float | None = None):
    obs = observations.dropna(subset=['meanRatio', 'observedFlowVph']).copy()
    obs = obs[obs['meanRatio'] > 1.0]
    if obs.empty:
        return None

    flows = obs['observedFlowVph'].values
    ratios = obs['meanRatio'].values
    capacity_guess = initial_capacity
    if capacity_guess is None:
        capacity_guess = np.nanmean(obs.get('capacityGuessVph', np.nan))
    if not np.isfinite(capacity_guess) or capacity_guess <= 0:
        capacity_guess = flows.max() * 1.25

    def residuals(params: np.ndarray) -> np.ndarray:
        alpha, beta, capacity = params
        if alpha <= 0 or beta <= 0 or capacity <= 0:
            return np.ones_like(ratios) * 1e3
        modeled = 1.0 + alpha * (flows / capacity) ** beta
        return modeled - ratios

    x0 = np.array([initial_alpha, initial_beta, capacity_guess], dtype=float)
    bounds = (np.array([1e-6, 0.5, 100.0]), np.array([5.0, 12.0, 20000.0]))
    result = least_squares(residuals, x0=x0, bounds=bounds)

    alpha, beta, capacity = result.x
    modeled = 1.0 + alpha * (flows / capacity) ** beta
    mae = np.mean(np.abs(modeled - ratios))
    rmse = np.sqrt(np.mean((modeled - ratios) ** 2))

    return {
        'alpha': float(alpha),
        'beta': float(beta),
        'capacityVph': float(capacity),
        'mae': float(mae),
        'rmse': float(rmse),
        'nObs': int(len(obs)),
        'maxObservedFlowVph': float(flows.max()),
    }


## Run calibration per segment
The loop below processes every segment that has both travel-time data and vehicle counts. Feel free to filter to a subset of segments if you want to focus on specific corridors first.


In [ ]:
calibration_rows = []
for segment_id, segment_obs in aligned.groupby('segmentId'):
    result = calibrate_bpr(segment_obs)
    if result is None:
        continue
    calibration_rows.append({
        'segmentId': segment_id,
        **result,
    })

calibrated = pd.DataFrame(calibration_rows)
calibrated.sort_values('rmse', inplace=True)
print(f'Segments calibrated: {len(calibrated)}')
calibrated


## Visual diagnostics
Select a segment to inspect observed vs. modelled ratios and flows. This helps confirm that the fitted curve matches your expectations.


In [ ]:
segment_to_plot = calibrated['segmentId'].iloc[0] if not calibrated.empty else None
if segment_to_plot:
    params = calibrated.set_index('segmentId').loc[segment_to_plot]
    subset = aligned[aligned['segmentId'] == segment_to_plot]
    flows = subset['observedFlowVph'].values
    ratios = subset['meanRatio'].values
    modeled = 1.0 + params['alpha'] * (flows / params['capacityVph']) ** params['beta']

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.scatter(flows, ratios, color='tab:blue', label='Observed mean ratio')
    flow_grid = np.linspace(0, max(flows.max(), params['capacityVph'] * 1.1), 100)
    model_grid = 1.0 + params['alpha'] * (flow_grid / params['capacityVph']) ** params['beta']
    ax.plot(flow_grid, model_grid, color='tab:red', label='BPR model fit')
    ax.set_xlabel('Flow (veh/h)')
    ax.set_ylabel('Travel-time ratio')
    ax.set_title(f'BPR calibration – {segment_to_plot}')
    ax.legend()
    ax.grid(True, which='both', alpha=0.3)
else:
    print('No calibrated segments available to plot.')


## Export calibrated coefficients
The CSV written below captures the fitted \u03b1, \u03b2, and capacity values. Copy the relevant entries into `src/segments.js`, inside each segment's `metadata.flowModel` block (`{ alpha, beta }`). Also consider updating `laneCapacityVph`/`lanes` if the calibrated capacity differs meaningfully from the assumptions used so far.


In [ ]:
if not calibrated.empty:
    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
    calibrated.to_csv(OUTPUT_PATH, index=False)
    display(calibrated)
    print(f'Calibration table written to: {OUTPUT_PATH}')
else:
    print('No calibration results to export.')


---
### Next steps
1. Review the `rmse`/`mae` values – high errors may indicate that the counts do not align with the sampled travel times or that the BPR form is not adequate for that segment.
2. Copy the calibrated parameters into `src/segments.js`, for example:
   ```js
   metadata: {
     lanes: 1,
     laneCapacityVph: 900,
     flowModel: { alpha: 0.27, beta: 3.6 }
   }
   ```
3. Re-run `npm run enrich` so the JSONL dataset picks up the new coefficients.
4. Commit the updated `src/segments.js` and the exported calibration CSV (if you want to archive the results).
